# データ・サイエンス	データ・サイエンス社会応用論 / ICT社会応用演習Ⅳ
## 14-1. 防災データを触ってみよう

### 1. 気象庁防災情報XMLの読み込み
#### 1-a) 下準備

In [1]:
import xml.etree.ElementTree as ET
from dateutil import parser
import requests
import pandas as pd

In [2]:
REGURLAR_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/regular.xml'
EXTRA_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/extra.xml'
EQVOL_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/eqvol.xml'
OTHER_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/other.xml'

REGURLAR_LONG_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/regular_l.xml'
EXTRA_LONG_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/extra_l.xml'
EQVOL_LONG_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/eqvol_l.xml'
OTHER_LONG_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/other_l.xml'

#### 1-b) フィードのダウンロード

In [3]:
response = requests.get(REGURLAR_FEED)
response.encoding = 'utf-8'
print(response.text)

<?xml version="1.0" encoding="utf-8"?>
<feed xmlns="http://www.w3.org/2005/Atom" lang="ja">
  <title>高頻度（定時）</title>
  <subtitle>JMAXML publishing feed</subtitle>
  <updated>2025-01-22T16:44:09+09:00</updated>
  <id>https://www.data.jma.go.jp/developer/xml/feed/regular.xml#short_1737531849</id>
  <link rel="related" href="https://www.jma.go.jp/"/>
  <link rel="self" href="https://www.data.jma.go.jp/developer/xml/feed/regular.xml"/>
  <link rel="hub" href="http://alert-hub.appspot.com/"/>
  <rights type="html"><![CDATA[
<a href="https://www.jma.go.jp/jma/kishou/info/coment.html">利用規約</a>,
<a href="https://www.jma.go.jp/jma/en/copyright.html">Terms of Use</a>
]]></rights>
  <entry>
    <title>警報級の可能性（明後日以降）</title>
    <id>https://www.data.jma.go.jp/developer/xml/data/20250122074315_0_VPFW60_472000.xml</id>
    <updated>2025-01-22T07:43:11Z</updated>
    <author>
      <name>南大東島地方気象台</name>
    </author>
    <link type="application/xml" href="https://www.data.jma.go.jp/developer/xml/dat

#### 1-c) DataFrameの作成

In [4]:
NS = {'': 'http://www.w3.org/2005/Atom'}

root = ET.fromstring(response.content)

rows = []
for entry in root.findall('entry', NS):
    rows.append({
        'title' : entry.find('title', NS).text,
        'content' : entry.find('content', NS).text,
        'updated' : parser.parse(entry.find('updated', NS).text),
        'link' : entry.find('link', NS).get('href')
    })

df_feed = pd.DataFrame(rows)
df_feed

,title,content,updated,link
0,警報級の可能性（明後日以降）,【大東島地方警報級の可能性（明後日以降）】,2025-01-22 07:43:11+00:00,https://www.data.jma.go.jp/developer/xml/data/...
1,警報級の可能性（明後日以降）,【宮古島地方警報級の可能性（明後日以降）】,2025-01-22 07:43:11+00:00,https://www.data.jma.go.jp/developer/xml/data/...
2,警報級の可能性（明後日以降）,【八重山地方警報級の可能性（明後日以降）】,2025-01-22 07:43:11+00:00,https://www.data.jma.go.jp/developer/xml/data/...
3,警報級の可能性（明後日以降）,【山口県警報級の可能性（明後日以降）】,2025-01-22 07:43:11+00:00,https://www.data.jma.go.jp/developer/xml/data/...
4,警報級の可能性（明後日以降）,【福岡県警報級の可能性（明後日以降）】,2025-01-22 07:43:11+00:00,https://www.data.jma.go.jp/developer/xml/data/...
...,...,...,...,...
635,府県天気概況,【天気概況】,2025-01-21 22:13:06+00:00,https://www.data.jma.go.jp/developer/xml/data/...
636,府県天気予報（Ｒ１）,【八重山地方府県天気予報】,2025-01-21 22:10:40+00:00,https://www.data.jma.go.jp/developer/xml/data/...
637,大雨危険度通知,【大雨危険度通知】,2025-01-21 22:07:24+00:00,https://www.data.jma.go.jp/developer/xml/data/...
638,大雨危険度通知,【大雨危険度通知】,2025-01-21 21:57:23+00:00,https://www.data.jma.go.jp/developer/xml/data/...


#### 1-d) タイトルで集計

In [5]:
df_feed.groupby('title')[['title']].count()

,title
title,
アジア太平洋地上実況図,1
アジア太平洋海上悪天２４時間予想図,1
アジア太平洋海上悪天４８時間予想図,1
全般季節予報（２週間気温予報）,1
地上実況図,3
地上２４時間予想図,1
地上４８時間予想図,1
地方季節予報（２週間気温予報）,1
大雨危険度通知,60


### 2. 地震エントリの読み込み
#### 2-a) エントリーのダウンロード

In [6]:
EQ_ENTRY = 'https://www.data.jma.go.jp/developer/xml/data/20250111214223_0_VXSE53_010000.xml' 

In [7]:
response = requests.get(EQ_ENTRY)
response.encoding = 'utf-8'
print(response.text)

<?xml version="1.0" encoding="UTF-8"?>
<Report xmlns="http://xml.kishou.go.jp/jmaxml1/" xmlns:jmx="http://xml.kishou.go.jp/jmaxml1/">
<Control>
<Title>震源・震度に関する情報</Title>
<DateTime>2025-01-11T21:42:23Z</DateTime>
<Status>通常</Status>
<EditorialOffice>気象庁本庁</EditorialOffice>
<PublishingOffice>気象庁</PublishingOffice>
</Control>
<Head xmlns="http://xml.kishou.go.jp/jmaxml1/informationBasis1/">
<Title>震源・震度情報</Title>
<ReportDateTime>2025-01-12T06:42:00+09:00</ReportDateTime>
<TargetDateTime>2025-01-12T06:42:00+09:00</TargetDateTime>
<EventID>20250112063948</EventID>
<InfoType>発表</InfoType>
<Serial>1</Serial>
<InfoKind>地震情報</InfoKind>
<InfoKindVersion>1.0_1</InfoKindVersion>
<Headline>
<Text>１２日０６時３９分ころ、地震がありました。</Text>
</Headline>
</Head>
<Body xmlns="http://xml.kishou.go.jp/jmaxml1/body/seismology1/" xmlns:jmx_eb="http://xml.kishou.go.jp/jmaxml1/elementBasis1/">
<Earthquake>
<OriginTime>2025-01-12T06:39:00+09:00</OriginTime>
<ArrivalTime>2025-01-12T06:39:00+09:00</ArrivalTime>
<Hypocenter>


#### 2-b) 情報の取得

In [8]:
NS = {
    '': 'http://xml.kishou.go.jp/jmaxml1/body/seismology1/',
    'jmx_eb': 'http://xml.kishou.go.jp/jmaxml1/elementBasis1/'
}

root = ET.fromstring(response.content)
earthquake = root.find('Body', NS).find('Earthquake', NS)
time = parser.parse(earthquake.find('OriginTime', NS).text)
area = earthquake.find('Hypocenter', NS).find('Area', NS)
area_name = area.find('Name', NS).text
area_coord = area.find('jmx_eb:Coordinate', NS).text
magnitude = earthquake.find('jmx_eb:Magnitude', NS).text

print(time, area_name, area_coord, magnitude)

2025-01-12 06:39:00+09:00 山形県置賜地方 +37.8+140.1-10000/ 2.9
